# Creating a micro-level cost comparison dataset

A significant challenge in the review of capital projects, especially in recent years, has been verifying inflation-induced cost escalations. In the absence of good, organized data on historical actual project costs, it is challenging to evaluate the reasonableness of cost escalations. However, if we can compare current project requests to prior, similar projects' budgets -- alongside CPI and PPI data showing inflation rates -- we can ask better questions about why cost escalations are occurring.

The code below is a replicable script that loops through our team's comprehensive records on capital project budgets, pulls out the cost estimate data, and builds a large, line-item-level dataset of line-item unit costs. It also extracts key metadata about the project, including the date of the project (allowing us to track inflation) and the folder location of the project. 

**NOTE:** While this script does the extraction of budget data, there are many other factors to consider to do a proper cost comparison across projects. Outside of this code, I have written scripts to identify other key metadata about projects, especially regarding the type of scope the project is pursuing and the history of the project (i.e., whether a project request builds upon a prior project request). This is important to consider given that concrete for a pier project may be different from concrete for an upland industrial building project. This code generates the "barebones" dataset, while other scripts add more detail to improve and ease analysis.

# 1. Import Packages

In [ ]:
import pandas as pd
import numpy as np
import os
from openpyxl import load_workbook
from datetime import datetime
import pytz
import re
import xlrd


# 2. Scrape and Extract Excel Cost Estimates

A. Scrape cost estimates in "Submission" Folder: That is where they are being deposited.
        
C. Extract budget data, creating dataset of line-items for each project

D. Append this to a larger dataset

**Key information from cost estimate**
- Item (columns[1])
- Quantity (columns[2])
- Units (columns[3])
- Unit cost (columns[4])
- Total cost (columns[5])

## 2. Scrape cost estimates not in "Submission" Folder

In [ ]:
## For CEs that *ARE NOT* in the submission folder

cost_estimates=[]

def find_and_read_cost_estimates(root_folder, keywords):
    
    processed_files=[]
    
    ## initialize dictionary with all our data
    while True:
        
        try:

            for foldername, subfolders, filenames in os.walk(root_folder):

                ## Subfolder where CE is will likely be called "Submission"

                if ' ' in foldername.lower():
                    
                    for filename in filenames:

                        ## looking for excel file

                        if filename.endswith('.xlsx') or filename.endswith('.xls'):
                            
                            if any(keyword.lower() in filename.lower() for keyword in keywords):
                            
                                if filename in processed_files:

                                    continue

                                file_path=os.path.join(foldername, filename)

                                #### need to also make sure we do not process files whose cost estimates have already been processed

                                if file_path in completed_ces:

                                    continue

                                engine='openpyxl' if filename.endswith('.xlsx') else 'xlrd'

                                with pd.ExcelFile(file_path,engine=engine) as xls:
                                    for sheet_name in xls.sheet_names:

                                        df=pd.read_excel(file_path, sheet_name)

                                        timestamp=datetime.utcfromtimestamp(os.path.getmtime(file_path))

                                        ## get entire cost estimate

                                        subfolder_parts = foldername.split(os.path.sep)
                                        Project_Name = file_path

                                        df=df.dropna(axis=1, how='all')

                                        CE=df

                                        ## get lists for materials, units, quantities, unit prices, total prices

                                        Materials=np.array(df[df.columns[0]])
                                        Units=np.array(df[df.columns[2]])
                                        Unit_Price=np.array(df[df.columns[3]])
                                        #Total_Price=list(df[df.columns[6]])
                                        Quantity=np.array(df[df.columns[1]])
                                        
                                        ## FOR BNYDC it is Item, Unit Cost, Quantity, Unit
                                        ## FOR EDC it is Item, Quantity, Unit, Unit Cost

                                        ## Create and clean up more condensed CE
                                        
                                        ## NOTE: THIS WILL GET RID OF ANY ROW THAT HAS A NAN VALUE IN ANY OF
                                        ## THE FIRST FOUR COLUMNS - WHICH INCLUDES MANY OF THE SOFT COST LINE-ITEMS,
                                        ## AS WELL AS SOME COST ESTIMATES THAT ARE FORMATTED SUCH THAT THE COLUMNS
                                        ## TO THE RIGHT OF THE LINE-ITEM ARE BLANK BEFORE LISTING THE LINE-ITEM DATA.

                                        Materials_Units_Prices=pd.DataFrame(list([Materials, Units, Quantity,
                                                                                                             Unit_Price])).transpose().dropna()

                                        #cost_estimates['Materials, Units, Prices'][4]=cost_estimates['Materials, Units, Prices'][2]*cost_estimates['Materials, Units, Prices'][3]
                                        #Materials_Units_Prices=Materials_Units_Prices.rename(columns={0:'Item',1:'Unit',2:'Quantity',3:'Unit Cost'})
                                        
                                        Materials_Units_Prices=Materials_Units_Prices[1:]

                                        ## Project name will (most likely) be the third entry in the second column [may need a better way of identifying this]

                                        ## Convert UTC raw time to EST
                                        
                                        Time_Submitted=pytz.timezone('UTC').localize(timestamp).astimezone(pytz.timezone('America/New_York'))

                                        cost_estimates.append({'Project_Name': Project_Name,
                                                                      'Whole_DF':CE,
                                                                      'Cleaned_CE':Materials_Units_Prices,
                                                                      'Time_Submitted':Time_Submitted})

                                        processed_files.append(filename)  # Mark the file as processed


                                        print(Project_Name)
                                        print(foldername)


        except Exception as e:

                print(f"Error: {e}")
                print(Project_Name)
                processed_files.append(filename)


root_folder='NYC Office of Management and Budget\grp_HED - Documents\Econdev\ECONDEV CPs\CPs - TGI'

## Cost estimate could have one of many names
keywords=['Estimate', 'Budget', 'estimate', 'budget', "CE"]

find_and_read_cost_estimates(root_folder, keywords)

## FOR BNYDC it is Item, Unit Cost, Quantity, Unit
## FOR EDC it is Item, Quantity, Unit, Unit Cost

for i in range(len(cost_estimates)):
    cost_estimates[i]['Cleaned_CE'][4]=cost_estimates[i]['Project_Name']
    cost_estimates[i]['Cleaned_CE'][5]=cost_estimates[i]['Time_Submitted']

indices = [x for x in range(len(cost_estimates))]

items = [list(cost_estimates[x]['Cleaned_CE'][0]) for x in range(len(cost_estimates))]
items = [item for items in items for item in items]

unit_names = [list(cost_estimates[x]['Cleaned_CE'][2]) for x in range(len(cost_estimates))]
unit_names = [item for unit_names in unit_names for item in unit_names]

quantities = [list(cost_estimates[x]['Cleaned_CE'][1]) for x in range(len(cost_estimates))]
quantities = [item for quantities in quantities for item in quantities]

unit_prices = [list(cost_estimates[x]['Cleaned_CE'][3]) for x in range(len(cost_estimates))]
unit_prices = [item for unit_prices in unit_prices for item in unit_prices]

times = [list(cost_estimates[x]['Cleaned_CE'][5]) for x in range(len(cost_estimates))]
times = [item for times in times for item in times]

project_names = [list(cost_estimates[x]['Cleaned_CE'][4]) for x in range(len(cost_estimates))]
project_names = [item for project_names in project_names for item in project_names]

total_data_list=[[items[x],unit_names[x],quantities[x],
                  unit_prices[x],times[x],project_names[x]] for x in range(len(items))]

total_dataframe=pd.DataFrame(total_data_list)
total_dataframe=total_dataframe.drop_duplicates()
total_dataframe=total_dataframe.rename(columns={0:'Item',
                                               1:'Quantity',
                                               2:'Unit',
                                               3:'Unit Price',
                                               4:'Time Approved',
                                               5: 'Name of Project'})

total_dataframe['Time Approved']=total_dataframe['Time Approved'].dt.tz_localize(None)

total_dataframe.to_excel('Downloads\Full_Line_Item_Spreadsheet_BNYDC.xlsx')